In [2]:
from __future__ import print_function
from six.moves import range
from PIL import Image

import torch.backends.cudnn as cudnn
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import os
import time

import sys
sys.path.append('/home/michaelcai/GANtor-Arts-Center/src/code/')

import numpy as np
import torchfile

from miscc.config import cfg, cfg_from_file
from miscc.utils import mkdir_p
from miscc.utils import weights_init
from miscc.utils import save_img_results, save_model
from miscc.utils import KL_loss
from miscc.utils import compute_discriminator_loss, compute_generator_loss

from tensorboard import summary
from tensorboardX import FileWriter

import torchvision.utils as vutils

In [3]:
import pandas as pd
import csv

In [4]:
csv_path = '/home/michaelcai/GANtor-Arts-Center/data/genre_train.csv'
filenames, classes = [], []
with open(csv_path) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        filenames.append(str(row[0])) 
        classes.append(int(row[1]))


In [5]:
folders = {0: 'abstract_painting', 
           1: 'cityscape', 
           2: 'genre_painting', 
           3: 'illustration', 
           4: 'landscape', 
           5: 'nude_painting', 
           6: 'portrait',
           7: 'religious_painting',
           8: 'sketch_and_study',
           9: 'still_life'}
print(folders)

{0: 'abstract_painting', 1: 'cityscape', 2: 'genre_painting', 3: 'illustration', 4: 'landscape', 5: 'nude_painting', 6: 'portrait', 7: 'religious_painting', 8: 'sketch_and_study', 9: 'still_life'}


In [6]:
classnums = [0] * 10

In [12]:
import subprocess
for i in range(len(filenames)):
    img_path = '/home/michaelcai/GANtor-Arts-Center/data/wikiart/' + filenames[i]
    subprocess.call(['cp', img_path, "./test.jpg"])
    img_name = filenames[i][filenames[i].rfind('/'):]
    save_path = '/home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/' + folders[classes[i]] + img_name
    subprocess.call(['cp', img_path, save_path])
    if i % 100 == 0:
        print("finished iteration " + str(i) + "/" + str(len(filenames)))
        print("Saved " + save_path)
    classnums[classes[i]] += 1

finished iteration 0/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/religious_painting/hieronymus-bosch_st-jacques-and-the-magician-hermogenes.jpg
finished iteration 100/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/religious_painting/fernando-botero_the-dolorosa.jpg
finished iteration 200/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/landscape/alfred-sisley_village-on-the-banks-of-the-seine-1872.jpg
finished iteration 300/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/portrait/gwen-john_young-woman-wearing-a-large-hat.jpg
finished iteration 400/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/landscape/albert-marquet_le-jardin-du-luxembourg.jpg
finished iteration 500/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/abstract_painting/conrad-marca-relli_untitled-1960.jpg
finished iteration 600/64995
Saved /home/michaelcai/G

finished iteration 5300/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/abstract_painting/sam-francis_untitled-1992.jpg
finished iteration 5400/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/portrait/mikhail-vrubel_portrait-of-a-businessman-k-artsybushev.jpg
finished iteration 5500/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/religious_painting/hans-memling_virgin-and-child-in-a-landscape.jpg
finished iteration 5600/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/genre_painting/childe-hassam_old-dutch-building-fishkill-new-york.jpg
finished iteration 5700/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/genre_painting/carl-larsson_oto-o-1884(1).jpg
finished iteration 5800/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/genre_painting/alfred-stevens_the-japanese-mask.jpg
finished iteration 5900/64995
Saved /home/michaelcai/G

finished iteration 10500/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/nude_painting/jean-fautrier_nu-feminin-allong-1937.jpg
finished iteration 10600/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/landscape/willard-metcalf_a-family-of-birches.jpg
finished iteration 10700/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/religious_painting/ivan-milev_the-cross-1923.jpg
finished iteration 10800/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/landscape/michel-simonidy_marine-landscape-1900.jpg
finished iteration 10900/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/genre_painting/pierre-auguste-renoir_young-girl-combing-her-hair-1892.jpg
finished iteration 11000/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/genre_painting/olexandr-archipenko_woman-with-umbrella-1919.jpg
finished iteration 11100/64995
Saved /home/michaelcai/GA

finished iteration 15700/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/genre_painting/james-tissot_the-last-evening.jpg
finished iteration 15800/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/sketch_and_study/kazimir-malevich_the-athlete-of-the-future-1913.jpg
finished iteration 15900/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/landscape/claude-monet_morning-on-the-seine-1.jpg
finished iteration 16000/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/landscape/konstantin-bogaevsky_cimmerian-region-1910.jpg
finished iteration 16100/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/genre_painting/tsuguharu-foujita_femme-en-pri-re.jpg
finished iteration 16200/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/landscape/ivan-shishkin_oak-grove-1893.jpg
finished iteration 16300/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/co

finished iteration 20900/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/portrait/john-russell_micoc-and-tootac-1769.jpg
finished iteration 21000/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/nude_painting/anders-zorn_the-tub-1888.jpg
finished iteration 21100/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/genre_painting/childe-hassam_building-a-schooner-provincetown.jpg
finished iteration 21200/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/religious_painting/gustave-dore_elijah-ascends-to-heaven-in-a-chariot-of-fire.jpg
finished iteration 21300/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/genre_painting/gustave-courbet_poor-woman-of-the-village.jpg
finished iteration 21400/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/cityscape/konstantin-bogaevsky_dnieprostroy.jpg
finished iteration 21500/64995
Saved /home/michaelca

...

finished iteration 41300/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/abstract_painting/clyfford-still_1945-h-1945.jpg
finished iteration 41400/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/cityscape/maerten-van-heemskerck_hanging-gardens-of-babylon.jpg
finished iteration 41500/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/landscape/ion-tuculescu_landscape-with-tree.jpg
finished iteration 41600/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/landscape/nicholas-roerich_brahmaputra-1946.jpg
finished iteration 41700/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/genre_painting/katsushika-hokusai_a-scene-of-a-shinto-shrine-dance-kagura.jpg
finished iteration 41800/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/landscape/claude-monet_misty-morning-on-the-seine-mauve.jpg
finished iteration 41900/64995
Saved /home/michaelcai

finished iteration 46500/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/genre_painting/georges-seurat_man-painting-his-boat-1883.jpg
finished iteration 46600/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/genre_painting/ernst-ludwig-kirchner_two-girls.jpg
finished iteration 46700/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/genre_painting/henri-martin_poetesses-on-a-take-shore.jpg
finished iteration 46800/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/genre_painting/man-ray_silhouette.jpg
finished iteration 46900/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/genre_painting/konstantin-makovsky_genre-scene.jpg
finished iteration 47000/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/genre_painting/vasily-perov_fishers-priest-deacon-and-seminarian-1879.jpg
finished iteration 47100/64995
Saved /home/michaelcai/GANtor-Arts-C

finished iteration 51600/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/cityscape/mstislav-dobuzhinsky_vilnius-nocturnal-scene.jpg
finished iteration 51700/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/cityscape/fernand-leger_at-the-farm-1955.jpg
finished iteration 51800/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/cityscape/camille-pissarro_avenue-de-l-opera-morning-sunshine-1898.jpg
finished iteration 51900/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/cityscape/juan-gris_landscape-at-beaulieu-1918.jpg
finished iteration 52000/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/cityscape/gustave-caillebotte_le-pont-de-l-europe-1.jpg
finished iteration 52100/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/cityscape/rudolf-von-alt_figures-on-the-riva-degli-schiavone-1840.jpg
finished iteration 52200/64995
Saved /home/micha

finished iteration 56900/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/still_life/georges-braque_two-mullets-1941.jpg
finished iteration 57000/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/still_life/josefa-de-obidos_still-life-1676.jpg
finished iteration 57100/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/still_life/martiros-saryan_still-life-1913-1.jpg
finished iteration 57200/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/still_life/constantin-piliuta_corn-poppies.jpg
finished iteration 57300/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/still_life/martiros-saryan_still-life-1959.jpg
finished iteration 57400/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/still_life/henri-fantin-latour_flowers-and-fruit-on-a-table-1865.jpg
finished iteration 57500/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_

finished iteration 61900/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/portrait/pablo-picasso_bust-of-woman-1906.jpg
finished iteration 62000/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/portrait/louise-elisabeth-vigee-le-brun_anna-ivanovna-tolstaya.jpg
finished iteration 62100/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/portrait/amedeo-modigliani_jeanne-hebuterne-with-white-collar-1919.jpg
finished iteration 62200/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/portrait/amedeo-modigliani_constant-leopold.jpg
finished iteration 62300/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/portrait/konstantin-makovsky_portrait-2.jpg
finished iteration 62400/64995
Saved /home/michaelcai/GANtor-Arts-Center/src/code/fid/Sorted_Images/portrait/pierre-auguste-renoir_graziella-1910.jpg
finished iteration 62500/64995
Saved /home/michaelcai/GANtor-Arts-Center/s

In [11]:
print(classnums)

[5153, 4764, 11217, 1958, 13780, 2001, 14675, 6783, 4092, 2900]


In [12]:
sum = 0
for cls in classnums:
    sum += cls
print(sum)

67323


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

